In [1]:
# 1.Libraries import and Data Loading
# ==========================================================
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
import joblib
import os

In [8]:
file_path = ("youtube_ad_revenue_dataset.csv")
df = pd.read_csv(file_path)
df




,video_id,date,views,likes,comments,watch_time_minutes,video_length_minutes,subscribers,category,device,country,ad_revenue_usd
0,vid_3092,2024-09-24 10:50:40.993199,9936,1221.0,320.0,26497.214184,2.862137,228086,Entertainment,TV,IN,203.178237
1,vid_3459,2024-09-22 10:50:40.993199,10017,642.0,346.0,15209.747445,23.738069,736015,Gaming,Tablet,CA,140.880508
2,vid_4784,2024-11-21 10:50:40.993199,10097,1979.0,187.0,57332.658498,26.200634,240534,Education,TV,CA,360.134008
3,vid_4078,2025-01-28 10:50:40.993199,10034,1191.0,242.0,31334.517771,11.770340,434482,Entertainment,Mobile,UK,224.638261
4,vid_3522,2025-04-28 10:50:40.993199,9889,1858.0,477.0,15665.666434,6.635854,42030,Education,Mobile,CA,165.514388
...,...,...,...,...,...,...,...,...,...,...,...,...
122395,vid_2902,2024-12-14 10:50:40.993199,9853,1673.0,147.0,42075.704885,25.490195,210818,Education,Tablet,US,280.986396
122396,vid_3890,2024-07-13 10:50:40.993199,10128,1709.0,63.0,57563.703040,16.229133,878860,Music,Desktop,UK,354.612981
122397,vid_3934,2024-06-10 10:50:40.993199,10267,700.0,NaN,27549.714659,23.822365,576756,Tech,Tablet,CA,203.643106
122398,vid_4260,2024-12-22 10:50:40.993199,10240,1616.0,106.0,56967.384382,7.753099,585138,Music,Mobile,UK,351.525811


In [10]:
# 2.Initial EDA

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122400 entries, 0 to 122399
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   video_id              122400 non-null  object 
 1   date                  122400 non-null  object 
 2   views                 122400 non-null  int64  
 3   likes                 116283 non-null  float64
 4   comments              116288 non-null  float64
 5   watch_time_minutes    116295 non-null  float64
 6   video_length_minutes  122400 non-null  float64
 7   subscribers           122400 non-null  int64  
 8   category              122400 non-null  object 
 9   device                122400 non-null  object 
 10  country               122400 non-null  object 
 11  ad_revenue_usd        122400 non-null  float64
dtypes: float64(5), int64(2), object(5)
memory usage: 11.2+ MB


In [11]:
df.describe()


,views,likes,comments,watch_time_minutes,video_length_minutes,subscribers,ad_revenue_usd
count,122400.000000,116283.000000,116288.000000,116295.000000,122400.000000,122400.000000,122400.000000
mean,9999.856283,1099.633618,274.396636,37543.827721,16.014165,502191.719902,252.727210
std,99.881260,519.424089,129.741739,12987.724246,8.083790,288397.470103,61.957052
min,9521.000000,195.000000,48.000000,14659.105562,2.000142,1005.000000,126.590603
25%,9933.000000,650.000000,162.000000,26366.320569,9.004695,252507.500000,199.902018
50%,10000.000000,1103.000000,274.000000,37531.990337,16.005906,503465.500000,252.749699
75%,10067.000000,1547.000000,387.000000,48777.782090,23.021260,752192.000000,305.597518
max,10468.000000,2061.000000,515.000000,61557.670089,29.999799,999997.000000,382.768254


In [12]:

df.isna().sum()


video_id                   0
date                       0
views                      0
likes                   6117
comments                6112
watch_time_minutes      6105
video_length_minutes       0
subscribers                0
category                   0
device                     0
country                    0
ad_revenue_usd             0
dtype: int64

In [13]:

df.columns

Index(['video_id', 'date', 'views', 'likes', 'comments', 'watch_time_minutes',
       'video_length_minutes', 'subscribers', 'category', 'device', 'country',
       'ad_revenue_usd'],
      dtype='object')

In [14]:
# Find row with highest ad revenue
idx_highest_ad_revenue = df['ad_revenue_usd'].idxmax()
row_highest_ad_revenue = df.loc[idx_highest_ad_revenue]
print(f"The row with the highest 'ad revenue' is:\n{row_highest_ad_revenue}")


The row with the highest 'ad revenue' is:
video_id                                  vid_3708
date                    2024-11-07 10:50:40.993199
views                                        10198
likes                                       2028.0
comments                                     375.0
watch_time_minutes                    61157.975673
video_length_minutes                      5.067021
subscribers                                 528766
category                                      Tech
device                                     Desktop
country                                         UK
ad_revenue_usd                          382.768254
Name: 115067, dtype: object


In [15]:
# Find row with lowest ad revenue
idx_lowest_ad_revenue = df['ad_revenue_usd'].idxmin()
row_lowest_ad_revenue = df.loc[idx_lowest_ad_revenue]
print(f"The row with the lowest 'ad revenue' is:\n{row_lowest_ad_revenue}")


The row with the lowest 'ad revenue' is:
video_id                                  vid_2182
date                    2024-11-19 10:50:40.993199
views                                         9874
likes                                        241.0
comments                                      67.0
watch_time_minutes                      14815.7704
video_length_minutes                       3.53417
subscribers                                 400281
category                                     Music
device                                      Tablet
country                                         CA
ad_revenue_usd                          126.590603
Name: 71148, dtype: object


In [16]:
# 3.Data Cleaning and Removing Duplicates

for col in df.select_dtypes(include=[np.number]).columns:
    df[col].fillna(df[col].mean(), inplace=True)

df.drop_duplicates(inplace=True)
print(df.info())


<class 'pandas.core.frame.DataFrame'>
Index: 120000 entries, 0 to 122399
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   video_id              120000 non-null  object 
 1   date                  120000 non-null  object 
 2   views                 120000 non-null  int64  
 3   likes                 120000 non-null  float64
 4   comments              120000 non-null  float64
 5   watch_time_minutes    120000 non-null  float64
 6   video_length_minutes  120000 non-null  float64
 7   subscribers           120000 non-null  int64  
 8   category              120000 non-null  object 
 9   device                120000 non-null  object 
 10  country               120000 non-null  object 
 11  ad_revenue_usd        120000 non-null  float64
dtypes: float64(5), int64(2), object(5)
memory usage: 11.9+ MB
None


In [17]:
# 4. Feature Engineering

df["date"] = pd.to_datetime(df["date"], errors="coerce")
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["day"] = df["date"].dt.day
df["day_of_week"] = df["date"].dt.dayofweek

In [18]:
# Derived features
df["engagement_rate"] = (df["likes"] + df["comments"]) / (df["views"])
df["average_watch_time"] = df["watch_time_minutes"] / (df["views"])
df["subscriber_ratio"] = df["views"] / (df["subscribers"])

df.drop(["video_id", "date"], axis=1, inplace=True)

In [19]:
# 5. Encode & Split

df = pd.get_dummies(df, columns=["category", "device", "country"], drop_first=True, dtype=int)

X = df.drop("ad_revenue_usd", axis=1)
print(X)
y = df["ad_revenue_usd"]
print(y)

        views   likes    comments  watch_time_minutes  video_length_minutes  \
0        9936  1221.0  320.000000        26497.214184              2.862137   
1       10017   642.0  346.000000        15209.747445             23.738069   
2       10097  1979.0  187.000000        57332.658498             26.200634   
3       10034  1191.0  242.000000        31334.517771             11.770340   
4        9889  1858.0  477.000000        15665.666434              6.635854   
...       ...     ...         ...                 ...                   ...   
122395   9853  1673.0  147.000000        42075.704885             25.490195   
122396  10128  1709.0   63.000000        57563.703040             16.229133   
122397  10267   700.0  274.396636        27549.714659             23.822365   
122398  10240  1616.0  106.000000        56967.384382              7.753099   
122399   9931   770.0  274.396636        38466.837135             27.305703   

        subscribers  year  month  day  day_of_week 

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [21]:
# 6. Models

models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(alpha=1.0),
    "Random Forest": RandomForestRegressor(n_estimators=100, max_depth=12, random_state=42, n_jobs=-1),
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=100,learning_rate=0.1, random_state=42),
    "Decision Tree": DecisionTreeRegressor(random_state=42)
}

results = {}
best_score = -np.inf
best_model = None
best_model_name = None

In [22]:
for name, model in models.items():
    print("=" * 60)
    print(f"Training {name}...")
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_true = y_test
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    r2_rounded = round(r2, 4)

    results[name] = {"MSE": f"{mse:.2f}", "MAE": f"{mae:.2f}", "R2": f"{r2:.4f}"}
    print(f"{name}: MSE={mse:.2f}, MAE={mae:.2f}, R²={r2:.4f}")

Training Linear Regression...
Linear Regression: MSE=181.70, MAE=3.12, R²=0.9526
Training Ridge Regression...
Ridge Regression: MSE=181.70, MAE=3.12, R²=0.9526
Training Random Forest...
Random Forest: MSE=184.28, MAE=3.53, R²=0.9519
Training Gradient Boosting...
Gradient Boosting: MSE=183.03, MAE=3.63, R²=0.9522
Training Decision Tree...
Decision Tree: MSE=373.67, MAE=5.29, R²=0.9025


In [23]:
if r2_rounded > best_score:
        best_score = r2_rounded
        best_model_name = name
        best_model = model

In [24]:
# 7. Save Artifacts

summary_df = pd.DataFrame(results).T
print("\n========== Summary of Models ==========")
print(summary_df)


========== Summary of Models ==========
                      MSE   MAE      R2
Linear Regression  181.70  3.12  0.9526
Ridge Regression   181.70  3.12  0.9526
Random Forest      184.28  3.53  0.9519
Gradient Boosting  183.03  3.63  0.9522
Decision Tree      373.67  5.29  0.9025


In [25]:
# Artifacts directory
ARTIFACTS_DIR = "model_artifacts"
os.makedirs(ARTIFACTS_DIR, exist_ok=True)
joblib.dump(scaler, os.path.join(ARTIFACTS_DIR, "scaler.pkl"))
joblib.dump(best_model, os.path.join(ARTIFACTS_DIR, f"{best_model_name}_model.pkl"))
joblib.dump(list(X.columns), os.path.join(ARTIFACTS_DIR, "training_columns.pkl"))
joblib.dump(results, os.path.join(ARTIFACTS_DIR, "results.pkl"))
print(f"Saved best model: {best_model_name}")


Saved best model: Decision Tree
